<a href="https://colab.research.google.com/github/sonmh79/VRPPD-with-robots/blob/main/R_VRPPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd

In [49]:
demand_data = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='demand',index_col=0)
d_data = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='OD_travel_time',index_col=0)
e3 = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='optimal_solution')

In [50]:
e1

,Pickup demand,Delivery demand
Customer ID,,
1,0,1
2,2,0
3,1,0
4,0,1
5,0,1
6,1,0
7,1,0
8,0,1
9,2,0


In [48]:
d_data

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00,65.69,85.69,219.38,244.38,253.58,294.72,334.72,283.92,262.78,293.12,358.12,302.32
1,65.69,0.00,64.55,158.24,183.24,192.44,233.58,273.58,222.78,201.64,231.98,296.98,241.18
2,85.69,64.55,0.00,218.24,243.24,252.44,293.58,333.58,282.78,261.64,291.98,356.98,301.18
3,219.38,158.24,218.24,0.00,59.55,137.10,178.24,218.24,167.44,146.30,176.64,241.64,185.84
4,244.38,183.24,243.24,59.55,0.00,162.10,203.24,243.24,192.44,171.30,201.64,266.64,210.84
5,253.58,192.44,252.44,137.10,162.10,0.00,125.69,84.55,183.24,162.10,192.44,257.44,201.64
6,294.72,233.58,293.58,178.24,203.24,125.69,0.00,125.69,224.38,203.24,233.58,298.58,242.78
7,334.72,273.58,333.58,218.24,243.24,84.55,125.69,0.00,264.38,243.24,273.58,338.58,282.78
8,283.92,222.78,282.78,167.44,192.44,183.24,224.38,264.38,0.00,65.69,204.38,269.38,213.58
9,262.78,201.64,261.64,146.30,171.30,162.10,203.24,243.24,65.69,0.00,183.24,248.24,192.44


In [51]:

class Rvrp(Robot):
  def __init__(self,demand_data,floor,n_robots):
    self.data = demand_data
    self.f = floor
    self.r = n_robots
    self.robots = []
    self._divide_floor()
  
  def _divide_floor(self):

    """ Initial Routes Construct """

    r = []
    for i in range(1,self.f+1):
      r.append(i)
      if len(r) == round(self.f / self.r):
        self.robots.append(r)
        r=[]
    if r:
      self.robots.append(r)

  def _is_delivery(self,node):
    return self.data.loc[node]["Delivery Demand"] != 0
  
  def _separate(self,robot):

    """ Separate Delivery with Pickup  """

    d,p= [],[]
    for node in robot:
      if self._is_delivery(node):
        d.append(node)
      else:
        p.append(node)
    return d + [0] + p
      
    




In [42]:
a = Rvrp(demand_data,12,4)
a.robots

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]